# Интегрирование функций

Постановка задачи: вычислить определенный интеграл 

$$
\int_a^{b} f(x)\, dx
$$

для данной функции $f(x)$ в заданных пределах от $a$ до $b$.

In [1]:
import numpy as np

%matplotlib notebook

import matplotlib.pyplot as plt
plt.xkcd()

In [2]:
def f(x):
    return (x - 1)**3 + 0.5

In [3]:
xx = np.linspace(-0.5, 2.5, 41)
plt.plot(xx, f(xx))

a, b = -0.4, 2.1

xxx = xx[(a < xx) & (xx < b)]
plt.fill_between(xxx, f(xxx), alpha=0.7, color='m')
plt.axvline(0, ls='--', lw=1); plt.axhline(0, ls='--', lw=1)

<IPython.core.display.Javascript object>

## Использование библиотеки SciPy

In [4]:
from scipy.integrate import quad

a, b = -0.4, 2.1
quad(f, a, b)

(0.6556250000000003, 1.9706427010952404e-14)


## Формулы Ньютона-Котеса

Рассмотрим разбиение интервала интегрирования

$$
a = x_0 < x_1 < \dots < x_{N_1} < x_N = b \;.
$$

На каждом *элементарном интервале*, $[x_j, x_{j+1}]$, аппроксимируем значение интеграла. Просуммировав результаты, получаем *составную формулу*, которая в пределе $N\to\infty$ дает точное значение интеграла.

Правило *левых прямоугольников*,


$$
\int_{x_0}^{x_0 + h} f(x)\, dx \approx f(x_0) \, h\;,
$$

является точным для кусочно-постоянных функций. Нетрудно показать, что погрешность составной формулы левых прямоугольников шкалируется линейно с размером шага разбиения, $\sim 1/N$.

In [5]:
def lrect(f, a, b, npts):
    h = (b - a) / npts
    summ = 0
    for j in range(npts):
        x = a + h*j
        summ += f(x) * h
    return summ

In [6]:
for n in (10, 100, 1000, 10000):
    res = lrect(lambda x: x**3, 0, 1, n)
    print("%5.5g : %g" % (n,  res - 0.25))

   10 : -0.0475
  100 : -0.004975
 1000 : -0.00049975
10000 : -4.99975e-05


Правило *средних прямоугольников*,

$$
\int_{x_0}^{x_0 + h} f(x)\,dx \approx f(x_0 + \frac{h}{2}) \, h\;,
$$

является точным для линейной функции $f(x)$, поэтому погрешность составной формулы $\sim 1/N^2$.

Заметим, что правило средних прямоугольников является примером *открытой* квадратурной формулы, не требующей вычисления значения подинтегральной функции на границах интервала интегрирования.

In [7]:
def crect(f, a, b, npts):
    h = (b - a) / npts
    summ = 0
    for j in range(npts):
        x = a + h*(j + 0.5)
        summ += f(x)
    return summ * h

In [8]:
for n in (10, 100, 1000, 10000):
    print("%5.5g : %g" % (n, crect(lambda x: x**3, 0, 1, n) - 0.25))

   10 : -0.00125
  100 : -1.25e-05
 1000 : -1.25e-07
10000 : -1.25e-09


Для построения квадратурных формул высших порядков рассмотрим элементарный интервал $[0, 1]$. (Заметим, что ограничения общности тут нет, т.к. любой интервал $[x_0, x_0 + h]$ линейным преобразованием приводится к интервалу $[0, 1]$).


Используем метод неопределенных коэффициентов: Представим значение интеграла как линейную комбинацию значений подынтегральной функции в $k+1$ точках, и подберем коэффициенты так, чтобы результат был бы точным для функций $x^0$, $x^1$, $\dots$, $x^k$. 

Например, построим трехточечную схему (т.е. *метод Симпсона*).

Положим 

$$
\int_0^1 f(x)\, dx = a f(x_0) + b f(x_1) + c f(x_2)\;,
$$
где $x_0 = 0$, $x_1 = \dfrac{1}{2}$, $x_2 = 1$ (т.е. $h=\dfrac{1}{2}$). 

Подберем коэффициенты $a$, $b$ и $c$ исходя из условий:

$$
\begin{matrix}
f(x) = 1 :& \qquad  &1            &= &a + &b + &c \;, \\
f(x) = x :& \qquad  &\dfrac{1}{2} &= &&\dfrac{1}{2}b + &c\;, \\
f(x) = x^2: &\qquad &\dfrac{1}{3} &= &&\dfrac{1}{4}b + &c \;.  
\end{matrix}
$$

В качестве решения имеем $a = c = \dfrac{1}{6}$ и $b = \dfrac{2}{3}$. Таким образом

$$
\begin{aligned}
\int_0^1 f(x) \,dx &\approx \frac{1}{6} \left( f_0 + 4 f_1 + f_2 \right) \\
                   &= \frac{1}{3} h \left(f_0 + 4f_1 + f_2 \right) \;.
\end{aligned}
$$

### Метод Ромберга


Предположим, что мы вычислили приближение $I_N$ интеграла $I$, такое что

$$
I_N = I + \gamma N^{-2} + \dots
$$


Тогда

$$
I^{(1)} = \frac{4 I_{2N} - I_N}{4 - 1}
$$

представляет собой улучшенное приближение точного значения интеграла $I$.

При вычислении производных мы эту технологию встречали под именем *экстраполяции Ричардсона*.

На этом месте большинство учебников останавливаются. А мы, наоборот, продолжим.

### Прежде чем приниматься вычислять интеграл, необходимо убедиться, что он существует

Например,

$$
\int_0^{\pi^2/4}\! \frac{dx}{\sin{x}}
$$

### Интегрируемые сингулярности


Например,

$$
I = \int_0^{\pi^2/4}\! \frac{dx}{\sin{\sqrt{x}}}
$$

Прибавим и вычтем член, имеющий такую же особенность на нижнем пределе интегрирования:

$$
\begin{aligned}
I &= \int_0^{\pi^2/4} \frac{dx}{\sin{\sqrt{x}}} \\
  &= \int_0^{\pi^2/4} \!dx \left( \frac{1}{\sin{\sqrt{x}}} - \frac{1}{\sqrt{x}} + \frac{1}{\sqrt{x}} \right) \\
  &= \int_0^{\pi^2/4} \!dx \left( \frac{1}{\sin{\sqrt{x}}} - \frac{1}{\sqrt{x}}  \right)  + \pi\;.
\end{aligned}
$$

Теперь оставшийся интеграл на нижнем пределе регулярен. Следует, однако, обратить внимание на возможную потерю точности при вычислении подынтегральной функции при малых значениях $x$.

## Адаптивные методы


Традиционно, методы Ньютона-Котеса формулируются на сетках с постоянным шагом. Если подынтегральная функция имеет резкие пики, равномерная сетка, очевидно, неоптимальна.

In [9]:
def f(x):
    return 1.0 / (1.0 + x)**3

In [10]:
a, b = 0, 3
xx = np.linspace(a, b, 21)
plt.plot(xx, f(xx))

### Простейшая идея: 

Используем метод средних прямоугольников, на каждом шаге разбиваем на две части прямоугольник максимальной площади.

In [11]:
import matplotlib.patches as patches

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(6, 10))

a, b = 0, 2
xx = np.linspace(a, b, 101)
ax1.plot(xx, f(xx))


# two rectangles
w, h = (a+b)/2., f((a+b)/4.)
ax1.add_patch(patches.Rectangle((0, 0), w, h, color='m', alpha=0.3, lw=5) )
ax1.add_patch(patches.Rectangle((w, 0), w, f(3/4*b), color='m', alpha=0.5, lw=5))
ax1.text(0.7, 0.7, 'Step 1.', transform=ax1.transAxes, fontsize=22)

# three rectangles
ax2.plot(xx, f(xx))
xs = [0, b/4, b/2]
ws = [b/4, b/4, b/2]
hs = [f(b/8), f(3*b/8), f(3*b/4)]
cs = ['g', 'g', 'm']
for j in [2, 1, 0]:
    ax2.add_patch(patches.Rectangle((xs[j], 0), ws[j], hs[j], color=cs[j], alpha=0.3, lw=5))
ax2.text(0.7, 0.7, 'Step 2.', transform=ax2.transAxes, fontsize=22)
    
# four rectangles:
ax3.plot(xx, f(xx))

xs = [0, b/8, b/4, b/2]
ws = [b/8, b/8, b/4, b/2]
hs = [f(b/16), f(b/8 + b/16), f(3*b/8), f(3*b/4)]
cs = ['r', 'r', 'g', 'm']
for j in [3, 2, 1, 0]:
    ax3.add_patch(patches.Rectangle((xs[j], 0), ws[j], hs[j], color=cs[j], alpha=0.3, lw=5))
ax3.text(0.7, 0.7, 'Step 3.', transform=ax3.transAxes, fontsize=22)

<IPython.core.display.Javascript object>

Text(0.7, 0.7, 'Step 3.')

In [12]:
# A rectangle is (start, width)
# A list element is (-weight, (start, width))

def make_rect(a, b, f):
    """Make a rectangle for the interval [a, b]"""
    rect = (a, b-a)
    xm = a + rect[1] / 2.
    return (-f(xm) * rect[1], rect)
    

def get_max_elem(lst, key=None):
    """Find and remove the maximum element from the list.
    
    Find the max element (according to the parameter `key`, which is a callable),
    remove it from the list, and return both the element and the rest.
    """
    if key is None:
        # use the identity function
        key = lambda x: x
    
    # find the max element
    elem = max(lst, key=key)
    
    # find its position in the list
    idx = lst.index(elem)
    
    return elem, lst[:idx] + lst[idx+1:]

    
def adapt_rect_list(f, a, b, npts):
    """Integrate f(x) from a to b using npts steps of the adaptive algorithm.
    """
    lst = []

    # start from a single rectangle
    item = make_rect(a, b, f)
    lst.append(item)
    
    # loop
    for _ in range(npts):
        # get the largest one
        rect, lst = get_max_elem(lst, lambda x: -x[0])
        w, (start, width) = rect
                
        # and split it into two halves
        c = start + width / 2.
        
        rect1 = make_rect(start, c, f)
        rect2 = make_rect(c, start + width, f)
        
        lst.append(rect1)
        lst.append(rect2)
        
    # collect the answer
    return -sum(w for w, r in lst), lst

In [13]:
def f(x):
    return np.exp(-x)
a, b = 0, 20

for n in (5, 25, 125, 250, 500, 1000, 5000):
    res, lst = adapt_rect_list(f, a, b, n)
    print ('n, res = ', n, res)


n, res =  5 0.9552698388347781
n, res =  25 0.9786718962428081
n, res =  125 0.9956502001607699
n, res =  250 0.9959491604791242
n, res =  500 0.9960027891786778
n, res =  1000 0.9997391610745809
n, res =  5000 0.999940521244624


In [14]:
% timeit adapt_rect_list(f, a, b, npts=1000)

UsageError: Line magic function `%` not found.


# Вопросы для практических занятий и самостоятельной работы:

1. Написать программу интегрирования заданной функции методом *правых* прямоугольников. Сравнить скорость сходимости метода с методами левых и средних прямоугольников.

2. Написать программу интегрирования заданной функции методом Симпсона. Сравнить скорость сходимости метода с методом средних прямоугольников.

1. Вычислить интеграл $\int_0^{\pi^2/4} dx \,/ \sin{\sqrt{x}}$ прямым вычислением и с вычитанием сингулярности. Обсудить скорость сходимости.

2. Разработать программу реализующую интегрирование заданной функции методом средних прямоугольников (или иным методом, на выбор) с адаптивной сеткой. Сравнить скорость сходимости данного метода и метода с равномерной сеткой.
*Указание: для выбора "наибольшего" интервала используйте модуль `heapq` из стандартной библиотеки. (При использовании языка `C++`, используйте `std::set`).* 

3. Вычислить зависимость плотности энергии идеального Ферми газа от температуры при фиксированной плотности частиц.

4. Вычислить зависимость теплоемкости идеального Бозе-газа от температуры. Обратить особое внимание на область температур вблизи области фазового перехода с появлением Бозе-конденсата.

5. При температурах ниже критических, изотермы газа Ван-дер-Ваальса содержат участки постоянного давления, $p(V)=\mathrm{const}$. Положение данных участков определяется согласно конструкции Максвелла, требующей равенства значений интегралов $\int p\,dV$ по участкам изотерм в области сосуществования фаз. Разработать программу, вычисляющую положения горизонтальных участков изотерм для газа Ван-дер-Ваальса.
